# CMAP Scan MUNE Algorithms: Analysis Notebook

This notebook provides a comprehensive guide to using the `STEPIX`, `CDIX`, and `Stairfit` algorithms for Motor Unit Number Estimation (MUNE) from CMAP scan data. 

**This workflow is divided into two main parts:**
1.  **Single Scan Analysis & Visualization:** We will load a single example `.MEM` file, run each of the three algorithms on the same data, and visualize the output. This is useful for understanding how each algorithm works.
2.  **Batch Processing:** We will use the `.MEF` files (which contain lists of `.MEM` files) to run the analysis on a full dataset of test/re-test scans. The results will be consolidated and saved to tidy CSV files for further statistical analysis.

## 1. Setup and Prerequisites

In [ ]:
import os
import sys
import pandas as pd
import numpy as np
import plotly.graph_objects as go

# Add the 'Scripts' directory to the Python path to allow importing our custom modules
sys.path.append('Scripts')

from Scripts.DataHandler import DataHandler
from Scripts.STEPIX import SETPIX
from Scripts.CDIX import CDIX
from Scripts.Stairfit import Stairfit

# Define directories
DATA_DIR = 'Data/CA-EDM 9'
RESULTS_DIR = 'Results'

# Create the results directory if it doesn't exist
os.makedirs(RESULTS_DIR, exist_ok=True)

print("Setup complete. Libraries and scripts imported successfully.")

## 2. Single Scan Analysis & Visualization

Here, we will analyze a single CMAP scan to demonstrate the output of each algorithm. This allows for a direct comparison of their results on the same underlying data.

In [ ]:
# --- Choose an example file to analyze ---
# We'll use the first file listed in the 'CA-EDM-MSF2 APB-1 9.MEF'
try:
    example_mef_file = os.path.join(DATA_DIR, 'CA-EDM-MSF2 APB-1 9.MEF')
    example_mem_list = DataHandler.get_mem_files_from_mef(example_mef_file)
    example_file_name = example_mem_list[0] # Select the first file from the list
    example_file_path = os.path.join(DATA_DIR, f"{example_file_name}.MEM")

    print(f"Selected example file: {example_file_path}")

    # Load the data using DataHandler
    scan_data = DataHandler(example_file_path)

except FileNotFoundError:
    print(f"Error: Make sure the example file exists at '{example_file_path}'")
    scan_data = None
except IndexError:
    print(f"Error: The MEF file '{example_mef_file}' appears to be empty.")
    scan_data = None

In [ ]:
# First, let's plot the raw CMAP scan data
if scan_data:
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=scan_data.x_trimmed, y=scan_data.y_trimmed, mode='markers', name='CMAP Data'))
    fig.update_layout(
        title=f'Raw CMAP Data for {scan_data.name}',
        xaxis_title='Stimulus Intensity',
        yaxis_title='CMAP Amplitude (mV)'
    )
    fig.show()

### 2.1. STEPIX Analysis

In [ ]:
if scan_data:
    # --- Algorithm Parameters ---
    stepix_threshold = 0.02 # Final threshold from manuscript
    
    # Run the algorithm
    stepix_results = SETPIX(scan_data.y_trimmed, NoiseThreshold=stepix_threshold, plot_results=True)

    print(f"--- STEPIX Results for {scan_data.name} ---")
    print(f"STEPIX: {stepix_results.STEPIX}")
    print(f"AMPIX: {stepix_results.AMPIX}")
    print(f"D50: {stepix_results.D50}")


### 2.2. CDIX Analysis

In [ ]:
if scan_data:
    # --- Algorithm Parameters ---
    cdix_threshold = 3 # Final threshold from manuscript

    # Run the algorithm
    # Note: CDIX uses untrimmed data.
    cdix_results = CDIX(scan_data.x, scan_data.y, Mean_LS=cdix_threshold, PlotResult=True)

    print(f"\n--- CDIX Results for {scan_data.name} ---")
    print(f"CDIX: {cdix_results.CDIX}")
    print(f"Number of Divisions: {cdix_results.NDivs}")
    print(f"Grid Size: {cdix_results.G}")

### 2.3. Stairfit Analysis

In [ ]:
if scan_data:
    # --- Algorithm Parameters ---
    stairfit_threshold = 0.015
    stairfit_mu_range = range(30, 200) # A reasonable search range for motor units

    # Run the algorithm
    # Note: Stairfit uses trimmed data.
    stairfit_results = Stairfit(scan_data.x_trimmed, scan_data.y_trimmed, 
                                termination_threshold=stairfit_threshold, 
                                MURange=stairfit_mu_range, 
                                Calc_Thres=True, 
                                PlotResult=True)

    print(f"\n--- Stairfit Results for {scan_data.name} ---")
    print(f"MUNE (M): {stairfit_results.M}")

## 3. Batch Processing and Saving Results

Now we will loop through all the `.MEM` files specified in the `.MEF` files for the test/re-test dataset. We'll run each algorithm and save the collated results into separate CSV files in the `Results` directory.

In [ ]:
# Define the MEF files to process. This dictionary maps a descriptive
mef_file_map = {
    'APB_V1': os.path.join(DATA_DIR, 'CA-EDM-MSF2 APB-1 9.MEF'),
    'APB_V2': os.path.join(DATA_DIR, 'CA-EDM-MSF2 APB-2 9.MEF'),
    'ADM_V1': os.path.join(DATA_DIR, 'CA-EDM-MSF2 ADM-1 9.MEF'),
    'ADM_V2': os.path.join(DATA_DIR, 'CA-EDM-MSF2 ADM-2 9.MEF'),
    'TA_V1': os.path.join(DATA_DIR, 'CA-EDM-MSF2 TA-1 9.MEF'),
    'TA_V2': os.path.join(DATA_DIR, 'CA-EDM-MSF2 TA-2 9.MEF'),
}
print("Beginning batch processing...")

### 3.1 CDIX Batch Analysis

In [ ]:
all_cdix_results = []

for condition, mef_path in mef_file_map.items():
    print(f"--- Running CDIX for: {condition} ---")
    mem_files = DataHandler.get_mem_files_from_mef(mef_path)
    for mem_file in mem_files:
        file_path = os.path.join(DATA_DIR, f"{mem_file}.MEM")
        try:
            data = DataHandler(file_path)
            # Use the same threshold as the single analysis
            results = CDIX(data.x, data.y, Mean_LS=cdix_threshold, PlotResult=False) # Plotting is off for batch processing
            all_cdix_results.append({
                'Condition': condition,
                'Filename': data.name,
                'CDIX': results.CDIX,
                'NDivs': results.NDivs,
                'Gridsize': results.G
            })
        except Exception as e:
            print(f"  Could not process {mem_file}. Error: {e}")

# Convert to DataFrame and save
cdix_df = pd.DataFrame(all_cdix_results)
cdix_output_path = os.path.join(RESULTS_DIR, 'batch_results_cdix.csv')
cdix_df.to_csv(cdix_output_path, index=False)

print(f"\nCDIX batch processing complete. Results saved to '{cdix_output_path}'")
display(cdix_df.head())

### 3.2 Stairfit Batch Analysis

In [ ]:
all_stairfit_results = []

for condition, mef_path in mef_file_map.items():
    print(f"--- Running Stairfit for: {condition} ---")
    mem_files = DataHandler.get_mem_files_from_mef(mef_path)
    for mem_file in mem_files:
        file_path = os.path.join(DATA_DIR, f"{mem_file}.MEM")
        try:
            data = DataHandler(file_path)
            results = Stairfit(data.x_trimmed, data.y_trimmed, 
                               termination_threshold=stairfit_threshold, 
                               MURange=stairfit_mu_range, 
                               Calc_Thres=False, 
                               PlotResult=False) # Plotting is off for batch processing
            all_stairfit_results.append({
                'Condition': condition,
                'Filename': data.name,
                'MUNE': results.M
            })
        except Exception as e:
            print(f"  Could not process {mem_file}. Error: {e}")

# Convert to DataFrame and save
stairfit_df = pd.DataFrame(all_stairfit_results)
stairfit_output_path = os.path.join(RESULTS_DIR, 'batch_results_stairfit.csv')
stairfit_df.to_csv(stairfit_output_path, index=False)

print(f"\nStairfit batch processing complete. Results saved to '{stairfit_output_path}'")
display(stairfit_df.head())

### 3.3 STEPIX Batch Analysis

In [ ]:
all_stepix_results = []

for condition, mef_path in mef_file_map.items():
    print(f"--- Running STEPIX for: {condition} ---")
    mem_files = DataHandler.get_mem_files_from_mef(mef_path)
    for mem_file in mem_files:
        file_path = os.path.join(DATA_DIR, f"{mem_file}.MEM")
        try:
            data = DataHandler(file_path)
            results = SETPIX(data.y_trimmed, NoiseThreshold=stepix_threshold)
            all_stepix_results.append({
                'Condition': condition,
                'Filename': data.name,
                'STEPIX': results.STEPIX,
                'AMPIX': results.AMPIX,
                'D50': results.D50
            })
        except Exception as e:
            print(f"  Could not process {mem_file}. Error: {e}")

# Convert to DataFrame and save
stepix_df = pd.DataFrame(all_stepix_results)
stepix_output_path = os.path.join(RESULTS_DIR, 'batch_results_stepix.csv')
stepix_df.to_csv(stepix_output_path, index=False)

print(f"\nSTEPIX batch processing complete. Results saved to '{stepix_output_path}'")
display(stepix_df.head())